# Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing Required Libraries

In [ ]:
import os
import torch
import wandb
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from PIL import Image
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd
from huggingface_hub import HfApi, HfFolder, Repository, create_repo, upload_file
from huggingface_hub import login
import torch.nn.functional as F
from timm.models.layers import trunc_normal_, DropPath
from timm.models.registry import register_model
from huggingface_hub import hf_hub_download
from skimage.metrics import structural_similarity as ssim
from glob import glob
import matplotlib.pyplot as plt

---

# Model Setup and Loading Checkpoints

In [ ]:
class CNNHead(nn.Module):
    def __init__(self):
        super(CNNHead, self).__init__()

        # Define the transposed convolution layers (upsampling)
        self.deconv1 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1)
        self.deconv4 = nn.ConvTranspose2d(32, 1, kernel_size=4, stride=2, padding=1)

        # Final convolution to adjust to the target size
        self.final_conv = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, x):

        CNNHead_features = []

        # Apply the transposed convolutions (upsampling)
        x = torch.relu(self.deconv1(x))
        CNNHead_features.append(x)

        x = torch.relu(self.deconv2(x))
        CNNHead_features.append(x)

        x = torch.relu(self.deconv3(x))
        CNNHead_features.append(x)

        x = self.deconv4(x)
        CNNHead_features.append(x)

        # Final convolution to adjust to the target size
        x = self.final_conv(x)
        CNNHead_features.append(x)

        # Add interpolation layer to achieve the target size
        x = F.interpolate(x, size=(600, 600), mode='bilinear', align_corners=False)  # [B, 1, 1151, 1151] -> [B, 1, 600, 600]
        CNNHead_features.append(x)


        return CNNHead_features

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, conv_channels=[64, 128, 256, 256]):
        super(ConvBlock, self).__init__()

        # Define the convolutional layers (feature extraction layers)
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels=conv_channels[0], kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=conv_channels[0], out_channels=conv_channels[1], kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=conv_channels[1], out_channels=conv_channels[2], kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=conv_channels[2], out_channels=conv_channels[3], kernel_size=3, stride=1, padding=1)

        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    def forward(self, x):
        ConvBlock_features = []

        # Apply the convolutional blocks sequentially
        x = self.pool(torch.relu(self.conv1(x)))
        ConvBlock_features.append(x)

        x = self.pool(torch.relu(self.conv2(x)))
        ConvBlock_features.append(x)

        x = self.pool(torch.relu(self.conv3(x)))
        ConvBlock_features.append(x)

        x = self.pool(torch.relu(self.conv4(x)))
        ConvBlock_features.append(x)


        # Return the feature map after the last convolutional layer
        return ConvBlock_features
        self.conv4 = nn.Conv2d(conv_channels[2], conv_channels[3], kernel_size=3, stride=1, padding=1)

        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    def forward(self, x):
        ConvBlock_features = []

        # Apply the convolutional blocks sequentially
        x = self.pool(torch.relu(self.conv1(x)))
        ConvBlock_features.append(x)

        x = self.pool(torch.relu(self.conv2(x)))
        ConvBlock_features.append(x)

        x = self.pool(torch.relu(self.conv3(x)))
        ConvBlock_features.append(x)

        x = self.pool(torch.relu(self.conv4(x)))
        ConvBlock_features.append(x)


        # Return the feature map after the last convolutional layer
        return ConvBlock_features

In [ ]:
class CNNEncoder(nn.Module):
    def __init__(self, in_channels=3, out_channels=32):
        super(CNNEncoder, self).__init__()

        # Conv1: [B, 3, 600, 600] -> [B, 16, 300, 300]
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)

        # Conv2: [B, 16, 300, 300] -> [B, 32, 600, 600]
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)

        # Activation function
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        CNNEncoder_features = []

        # Pass through conv1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        CNNEncoder_features.append(x)

        # Pass through conv2 (with upsampling)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        CNNEncoder_features.append(x)

        return CNNEncoder_features

In [ ]:
class AOHead(nn.Module):
    def __init__(self, in_channels=768, out_channels=1):
        super(AOHead, self).__init__()

        # Gradual upsampling with ConvTranspose2d layers
        self.deconv1 = nn.ConvTranspose2d(in_channels, 512, kernel_size=3, stride=2, padding=1, output_padding=1)  # [B, 768, 18, 18] -> [B, 512, 36, 36]
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1)         # [B, 512, 36, 36] -> [B, 256, 72, 72]
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1)         # [B, 256, 72, 72] -> [B, 128, 144, 144]
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1)          # [B, 128, 144, 144] -> [B, 64, 288, 288]
        self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1)           # [B, 64, 288, 288] -> [B, 32, 576, 576]

        # Activation functions
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()  # Normalize to range [0, 1]

    def forward(self, x):
        """
        Forward pass through the AO head.

        Parameters:
            x (torch.Tensor): The input feature map from the backbone.

        Returns:
            torch.Tensor: The predicted Ambient Occlusion map.
        """
        head_features = []

        # Gradual upsampling with ConvTranspose2d layers
        x = self.relu(self.deconv1(x))    # [B, 768, 18, 18] -> [B, 512, 36, 36]
        head_features.append(x)

        x = self.relu(self.deconv2(x))    # [B, 512, 36, 36] -> [B, 256, 72, 72]
        head_features.append(x)

        x = self.relu(self.deconv3(x))    # [B, 256, 72, 72] -> [B, 128, 144, 144]
        head_features.append(x)

        x = self.relu(self.deconv4(x))    # [B, 128, 144, 144] -> [B, 64, 288, 288]
        head_features.append(x)

        x = self.relu(self.deconv5(x))    # [B, 64, 288, 288] -> [B, 32, 576, 576]
        head_features.append(x)

        # Add interpolation layer to achieve the target size
        x = F.interpolate(x, size=(600, 600), mode='bilinear', align_corners=False)  # [B, 1, 1151, 1151] -> [B, 1, 600, 600]
        head_features.append(x)


        return head_features

In [ ]:
class Block(nn.Module):
    r""" ConvNeXt Block. There are two equivalent implementations:
    (1) DwConv -> LayerNorm (channels_first) -> 1x1 Conv -> GELU -> 1x1 Conv; all in (N, C, H, W)
    (2) DwConv -> Permute to (N, H, W, C); LayerNorm (channels_last) -> Linear -> GELU -> Linear; Permute back
    We use (2) as we find it slightly faster in PyTorch

    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim) # depthwise conv
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim) # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)),
                                    requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1) # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2) # (N, H, W, C) -> (N, C, H, W)

        x = input + self.drop_path(x)
        return x

In [ ]:
class ConvNeXt(nn.Module):
    r""" ConvNeXt
        A PyTorch impl of : `A ConvNet for the 2020s`  -
          https://arxiv.org/pdf/2201.03545.pdf
    Args:
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        depths (tuple(int)): Number of blocks at each stage. Default: [3, 3, 9, 3]
        dims (int): Feature dimension at each stage. Default: [96, 192, 384, 768]
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
        head_init_scale (float): Init scaling value for classifier weights and biases. Default: 1.
    """
    def __init__(self, in_chans=3, out_chans=1,
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0.,
                 layer_scale_init_value=1e-6, head_init_scale=1.,
                 **kwargs,):

        super().__init__()



        self.downsample_layers = nn.ModuleList() # stem and 3 intermediate downsampling conv layers
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = nn.Sequential(
                    LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)


        self.stages = nn.ModuleList() # 4 feature resolution stages, each consisting of multiple residual blocks
        dp_rates=[x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j],
                layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]



        # Output of AOHead part
        self.ao_head = AOHead(in_channels= dims[-1], out_channels=1)

        # Output of CNNEncoder part
        self.cnn_encoder = CNNEncoder(in_channels= 3, out_channels=32)

        # Output of ConvBlock part
        self.conv_block = ConvBlock(in_channels= 64, conv_channels=[64, 128, 256, 256])

        # Output of CNNHead part
        self.cnn_head = CNNHead()



        self.apply(self._init_weights)




    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)


    def forward_features(self, x):
        features = []
        input = x

        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
            features.append(x)

        # Output of AOHead part
        head_features = self.ao_head(x)
        features.extend(head_features)

        x = head_features[-1]


        # Output of CNNEncoder part
        cnn_encoder_features = self.cnn_encoder(input)
        features.extend(cnn_encoder_features)

        x = cnn_encoder_features[-1]


        # Output of ConvBlock part

        # Concatenate along the channel dimension (dim=1)
        concatenated_tensor = torch.cat((head_features[-1], cnn_encoder_features[-1]), dim=1)

        conv_block_features = self.conv_block(concatenated_tensor)
        features.extend(conv_block_features)

        x = conv_block_features[-1]

        # Output of CNNHead part
        CNNHead_features = self.cnn_head(x)
        features.extend(CNNHead_features)

        x = CNNHead_features[-1]


        return features


    def forward(self, x):
        features = self.forward_features(x)

        return features


In [ ]:
class LayerNorm(nn.Module):
    r""" LayerNorm that supports two data formats: channels_last (default) or channels_first.
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs
    with shape (batch_size, channels, height, width).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError
        self.normalized_shape = (normalized_shape, )

    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]

            return x

In [ ]:
model_urls = {
    "convnext_tiny_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth",
    "convnext_tiny_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_22k_224.pth",
    "AO_Model": "https://huggingface.co/prakanda/AO_Model/resolve/main/AO_Model.pth",
    "New_AO_Model_1": "https://huggingface.co/prakanda/New_AO_Model_1/resolve/main/New_AO_Model_1.pth",
 #   "ConvNeXt_ambient_occlusion_model_1" : "https://huggingface.co/prakanda/ConvNeXt_ambient_occlusion_model_1/resolve/main/ConvNeXt_ambient_occlusion_model_1.pth",

}

In [ ]:
def convnext_tiny(pretrained=True,load_ao=True,**kwargs):
    model = ConvNeXt(depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], **kwargs)
    if pretrained:
        #checkpoint = torch.load(kwargs['checkpoint'], map_location="cpu")

        if load_ao:
            url = model_urls['New_AO_Model_1']
            print("AO_Model has been loaded." )
        else:
            url = model_urls['convnext_tiny_1k']
            print("convnext_tiny_1k has been loaded")

        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)

        model_dict = model.state_dict()
        pretrained_dict = {}
        unmatched_pretrained_dict = {}

        for k, v in checkpoint.items():
            if k in model_dict:
                pretrained_dict[k] = v
            else:
                unmatched_pretrained_dict[k] = v
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)


        print(f'The type of checkpoint is {type(checkpoint)}.')
        print(f'The type of model.state_dict is {type(model.state_dict)}.')
        print(f'The type of model.state_dict() is {type(model.state_dict())}.')



        for name,param in model.named_parameters():
          if name in pretrained_dict.keys():
              param.requires_grad = True
          else :
              param.requires_grad = True


        print('\n')

        print(f'The keys in pretrained_dict are : \n {pretrained_dict.keys()}')

        print('\n')

        print(f'The keys in unmatched_pretrained_dict are : \n {unmatched_pretrained_dict.keys()}')

        print('\n')

        print(
            'Successfully loaded pretrained %d paras, and %d paras are unmatched.'
            %(len(pretrained_dict.keys()), len(unmatched_pretrained_dict.keys())))

        print('\n')

        print('Unmatched pretrained paras are:', unmatched_pretrained_dict.keys())

    return model

---

# Model Instantiation

In [ ]:
model = convnext_tiny(True, load_ao=True).cuda()

---

# Model Configuration Setup

In [ ]:
# Download the model file from Hugging Face Hub
repo_name = "New_AO_Model_1"
downloaded_file = hf_hub_download(
    repo_id=f"prakanda/{repo_name}",  # Replace with your Hugging Face username
    filename="New_AO_Model_1.pth"
)
print(f"Model downloaded from Hugging Face Hub: {downloaded_file}")


# Initialize the model and load the state_dict
model.load_state_dict(torch.load(downloaded_file),strict=False)

---

# Result Visualization

In [ ]:
# Define the transformation
transform = transforms.Compose([
        transforms.Resize((600, 600)),
        transforms.ToTensor(),          # Convert the image to a PyTorch tensor (HWC -> CHW, normalized [0, 1])
    ])


def path_to_tensor(image):
  image = Image.open(image).convert("RGB")
  tensor_image = transform(image)
  tensor_image = tensor_image.unsqueeze(0)
  return tensor_image

In [ ]:
def inference(path):
    input_tensor = path_to_tensor(path)
    input_tensor = input_tensor.to("cuda")

    # Example size, adjust as needed
    dic={"input":input_tensor}

    model.to("cuda")

    with torch.no_grad():  # No gradient calculation for evaluation/testing
        output = model(dic["input"])
        return output[-1]

In [ ]:
filenames=glob("/content/drive/MyDrive/AO/*")


# Assuming filenames is a list of file paths
for filepath in filenames:
    output = inference(filepath)
    image = output.to("cpu")

    # Squeeze batch and channel dimensions, then convert to numpy array
    ambient_image = image.squeeze(0).squeeze(0).numpy()

    # Load the original image from the file path
    original_image_path = Image.open(filepath)

    # Create a subplot with 2 images
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the original image (from the file)
    axs[0].imshow(original_image_path, cmap='gray')
    axs[0].set_title("Original Image from Path")
    axs[0].axis('off')  # Hide axes for better visualization

    # Plot the processed image (inverse normalized)
    axs[1].imshow(ambient_image, cmap='gray')
    axs[1].set_title("Ambient Occlusion Map")
    axs[1].axis('off')  # Hide axes for better visualization

    # Display the plot
    plt.tight_layout()
    plt.show()